# NLP segmentator

In [1]:
import re
import operator
from functools import reduce
import requests
import http.client
import json
from sklearn.feature_extraction.text import CountVectorizer
from enum import Enum
import xml.etree.ElementTree as ET
import pandas as pd
import ast
from tqdm import tqdm_notebook as tqdm
import numpy as np

In [2]:
def read_corpus_file(filename):
    def prepare_line_record(line):
        items = line.split('\t')
        sentence = items[1]
        label = items[0]
        return sentence, label
    lines = open(filename, "r",encoding='utf-8').read().splitlines()
    return [prepare_line_record(it) for it in lines]
train_corpus=read_corpus_file('text_source/plemo2.0-no-segm.txt')
train_corpus

[('Wykłady strasznie nudne, totalna porażka, nie warto przyłazic. Jeśli jednak zdecydujesz się chodzic na wykład to pamiętaj, że na zajęciach trzeba byc punktualnie, bo koleś zamyka drzwi ;) Koła w kilku salach, na każdej sali pilnuje kilka osób, ale ściagac da rade.',
  'meta_minus_m'),
 ('Ogólnie bardzo pozytywny choc troche zakręcony koleś ;) Zawsze przychodził w wygniecionych ciuchach, zupelnie jakby w nich spał po całonocnej popijawie. Koła standardowe - zadania jak ze skryptu, pełen luzik. Ogólnie polecam :) Aha - bardzo lubi udowadniac przez indukcje ;)',
  'meta_plus_s'),
 ('Pan Krzysztof to ogólnie bardzo pozytywna postać, na ćwiczeniach lubi się pośmiać (ze studentów i nie tylko). Zajęcia prowadzi w sposób ciekawy, potrafi przekazywać wiedzę. Koła zwykle do najłatwiejszych nie należą, ale jeśli człowiek się troszkę postara to zaliczy bez problemu.',
  'meta_plus_m'),
 ('Wiesław Dudek Analiza Matematyczna I Algebra Liniowa I Wymagający, ale spoko[/b] Prowadzący wzbudza mieszan

# Tokenizer

#### Split text to sentences

In [3]:
def split_text_to_sentences(text):
    text = text.replace('\n', ' ')
    sentences_list=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences_list]

#### Special tokens detection

In [12]:
def get_date_regex():
    r1=re.compile(r"[\d]{1,2} [slmkcśwpg]\w* [\d]{4}") #10 października 2019
    r2=re.compile(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}") # 10-10-2019
    r3=re.compile(r"[\d]{1,2}-[\d]{1,2}-[\d]{2}") #10-10-19
    r4=re.compile(r"[\d]{1,2}.[\d]{1,2}.[\d]{4}") #10.10.2019
    r5=re.compile(r"[\d]{1,2}.[\d]{1,2}.[\d]{2}") #10.10.19
    r6=re.compile(r"[\d]{1,2}/[\d]{1,2}/[\d]{4}") #10/10/2019
    r7=re.compile(r"[\d]{1,2}/[\d]{1,2}/[\d]{2}") #10/10/19
    regexes = [r1, r2, r3,r4,r5,r6,r7]
    # all date regexes in one
    return re.compile('|'.join(x.pattern for x in regexes))


def get_email_regex():
    return re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')


def get_time_regex():
    r1=re.compile(r"[2][0-3]:[0-5][0-9]")
    r2=re.compile(r"[0-1][0-9]:[0-5][0-9]")
    regexes = [r1, r2]
    return re.compile('|'.join(x.pattern for x in regexes))


def get_special_token_regex():
    return re.compile('|'.join(it.pattern for it in [get_date_regex(), get_email_regex(), get_time_regex()]))

#### Helper methods

In [5]:
def filter_special_token_regex(text_to_filter):
    pattern_combined = get_special_token_regex()
    return re.split('(' + pattern_combined.pattern + ')', text_to_filter)


def list_flatten(list_to_flatten):
    return [it for elem in list_to_flatten for it in elem]


def intersperse(lst, item):
    array = []
    for it in lst:
        array.append(it)
        array.append(item)
    array.pop()
    return array


def get_tokens(text_for_tokens):
    def split_with_token(value, delimeter):
        return intersperse(value.split(delimeter), delimeter)
    
    def support_non_special_token_sequence(sequence):
        characters_to_split = ['.', ',', ';']
        splitted = [it for it in sequence.split(' ') if len(it) > 0]
        for character in characters_to_split:
            splitted = list_flatten([split_with_token(it, character) for it in splitted])
        return [it for it in splitted if len(it) > 0]
    
    text_preprocessed = text_for_tokens.replace('\n', ' ').lower()
    special_token_extract_split = filter_special_token_regex(text_preprocessed)
    special_token_regex = get_special_token_regex()
    tokens_groupped = [
        [it] if special_token_regex.match(it) else support_non_special_token_sequence(it)
        for it in special_token_extract_split
    ]
    tokens_groupped_not_empty = [it for it in tokens_groupped if len(it) > 0]
    return list_flatten(tokens_groupped_not_empty)

def tokenize_formatted(text):
    result_string=''
    sentences=split_text_to_sentences(text)
    for sentence in sentences:
        result_string+=sentence
        result_string+='\n'
        tokens=get_tokens(sentence)
        for token in tokens:
            result_string=result_string+'['+token+']'
            result_string+='\n'
    return result_string   

### Sample results  -  novel "Katarynka" by  B. Prus

In [6]:
f = open("text_source/katarynka.txt", "r")
text=f.read()
sentences = split_text_to_sentences(text)
print(tokenize_formatted(text[0:200]))

Na ulicy Miodowej co dzień około południa można było spotkać jegomościa w pewnym wieku, który chodził z placu Krasińskich ku ulicy Senatorskiej.
[na]
[ulicy]
[miodowej]
[co]
[dzień]
[około]
[południa]
[można]
[było]
[spotkać]
[jegomościa]
[w]
[pewnym]
[wieku]
[,]
[który]
[chodził]
[z]
[placu]
[krasińskich]
[ku]
[ulicy]
[senatorskiej]
[.]
Latem nosił on wykwintne, ciemnogranatowe palto, popiel
[latem]
[nosił]
[on]
[wykwintne]
[,]
[ciemnogranatowe]
[palto]
[,]
[popiel]



### Example from corpus

In [7]:
print(tokenize_formatted(train_corpus[0][0]))

Wykłady strasznie nudne, totalna porażka, nie warto przyłazic.
[wykłady]
[strasznie]
[nudne]
[,]
[totalna]
[porażka]
[,]
[nie]
[warto]
[przyłazic]
[.]
Jeśli jednak zdecydujesz się chodzic na wykład to pamiętaj, że na zajęciach trzeba byc punktualnie, bo koleś zamyka drzwi ;) Koła w kilku salach, na każdej sali pilnuje kilka osób, ale ściagac da rade.
[jeśli]
[jednak]
[zdecydujesz]
[się]
[chodzic]
[na]
[wykład]
[to]
[pamiętaj]
[,]
[że]
[na]
[zajęciach]
[trzeba]
[byc]
[punktualnie]
[,]
[bo]
[koleś]
[zamyka]
[drzwi]
[;]
[)]
[koła]
[w]
[kilku]
[salach]
[,]
[na]
[każdej]
[sali]
[pilnuje]
[kilka]
[osób]
[,]
[ale]
[ściagac]
[da]
[rade]
[.]



### Example on special tokens

In [10]:
text_with_special_tokens="Dnia 12 lipca 1982, co właściwie mógłbym zapisać jako 12.07.1982, o godzinie 14:35 dostałem wiadomość na moją skrzynkę email od daniel.popek13@gmail.com"

In [11]:
print(tokenize_formatted(text_with_special_tokens))

Dnia 12 lipca 1982, co właściwie mógłbym zapisać jako 12.07.1982, o godzinie 14:35 dostałem wiadomość na moją skrzynkę email od daniel.popek13@gmail.com
[dnia]
[12 lipca 1982]
[,]
[co]
[właściwie]
[mógłbym]
[zapisać]
[jako]
[12.07.1982]
[,]
[o]
[godzinie]
[14:35]
[dostałem]
[wiadomość]
[na]
[moją]
[skrzynkę]
[email]
[od]
[daniel.popek13@gmail.com]



# Morphosyntactic analysis - Morfeusz API

In [170]:
def morfeusz_from_text(text):
    API_ENDPOINT = "http://ws.clarin-pl.eu/nlprest2/base/process"
    body_map = { 
        'lpmn':'any2txt|maca({"morfeusz2":true})',
        'text': text, 
        'user' : 'moj@adres.mail'
    }
    body_json = json.dumps(body_map)
    r = requests.post(url = API_ENDPOINT, data = body_json) 
    return r.text

In [171]:
def morfeusz_from_tokens(token_list):
    API_ENDPOINT = "http://ws.clarin-pl.eu/lexrest/lex"
    body_map = {
        'task':'all',
        'tool':'morfeusz',
        'lexeme':token_list
    }
    body_json = json.dumps(body_map)
    r = requests.post(url = API_ENDPOINT, data = body_json) 
    parsed = json.loads(r.text)
    return parsed['results']

In [172]:
def translate_morfeusz_result_to_list(results):
    result_list=[]
    for token in results.keys():
        parsing_list=results[token]['analyse']
        token_list=[(elem[0],elem[2]) for elem in parsing_list]
        result_list.append((token,token_list))
    return result_list

In [173]:
print(morfeusz_from_text('Lubię chodzić do kina na filmy z Katarzyną Kowalską.'))

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE chunkList SYSTEM "ccl.dtd">
<chunkList>
 <chunk id="ch1">
  <sentence id="s1">
   <tok>
    <orth>Lubię</orth>
    <lex><base>lubić</base><ctag>fin:sg:pri:imperf</ctag></lex>
   </tok>
   <tok>
    <orth>chodzić</orth>
    <lex><base>chodzić:v1</base><ctag>inf:imperf</ctag></lex>
    <lex><base>chodzić:v2</base><ctag>inf:imperf</ctag></lex>
   </tok>
   <tok>
    <orth>do</orth>
    <lex><base>do:p</base><ctag>prep:gen</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:nom:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:gen:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:dat:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:acc:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:inst:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:loc:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:sg:voc:n</ctag></lex>
    <lex><base>do:s</base><ctag>subst:pl:nom:n</ctag></lex>
    <lex><base>do:s</base><ctag>su